In [1]:
import numpy as np

In [3]:
N = 100
np.random.seed(0)
X = np.sort(5 * np.random.rand(N), axis=0)
y = np.sin(X)
y = y + 0.2 * np.random.randn(N)

In [5]:
import torch
import torch.nn as nn
X = torch.from_numpy(X).reshape(-1, 1).float()
y = torch.from_numpy(y).float()

In [6]:
class KernelSVR():
    def __init__(self, n_samples, eps=0.1, C=10, gamma=50, iters=50000, lr=1e-5):
        self.n_samples = n_samples
        self.eps = eps
        self.C = C
        self.gamma = gamma
        self.alpha_1 = nn.Parameter(torch.rand(n_samples))
        self.alpha_2 = nn.Parameter(torch.rand(n_samples))
        self.iters = iters
        self.lr = lr
    
    def _kernel_matrix(self, u, v):
        in_dim = u.shape[1]
        rbf_coeff = 1 / in_dim
        
        dist = torch.cdist(u, v) ** 2
        K = torch.exp(-rbf_coeff * dist)
        return K
    
    def _loss(self, K, y):
        loss1 = eps * (self.alpha_1 + self.alpha_2).sum()
        diff = self.alpha_1 - self.alpha_2
        loss2 = 0.5 * (torch.outer(diff, diff) * K).sum()
        loss3 = - torch.dot(diff, y)
        loss4 = self.gamma * diff.sum() ** 2
        loss = loss1 + loss2 + loss3 + loss4
        return loss
    
    def _zero_grad(self):
        if self.alpha_1.grad is not None:
            self.alpha_1.grad.zero_()
            self.alpha_2.grad.zero_()
    
    def _update_weights(self):
        self.alpha_1.data -= self.lr * self.alpha_1.grad.data
        self.alpha_2.data -= self.lr * self.alpha_2.grad.data
    
    def _clip_weights(self):
        self.alpha_1.data = torch.clamp(self.alpha_1.data, min=0, max=self.C)
        self.alpha_2.data = torch.clamp(self.alpha_2.data, min=0, max=self.C)
    
    def fit(self, X, y):
        K = self._kernel_matrix(X, X)
        for i in range(self.iters):
            self._zero_grad()
            self._clip_weights()
            loss = self._loss(K, y)
            loss.backward()
            self._update_weights()
        
        with torch.no_grad():
            self._clip_weights()
            self._X = X.clone()
            self._diff = (self.alpha_1 - self.alpha_2).data.clone()
            out = torch.matmul(self._diff, K)
            idx1 = torch.where(self.alpha_1>0)[0]
            b1 = (y[idx1] - out[idx1] - self.eps).min()
            idx2 = torch.where(self.alpha_2>0)[0]
            b2 = (y[idx2] - out[idx2] + self.eps).max()
            self.b = (b1 + b2)/2
    
    def predict(self, data):    
        with torch.no_grad():
            K = self._kernel_matrix(self._X, data)
            pred = torch.matmul(self._diff, K) + self.b
        
        return pred

In [12]:
eps = 0.5
C = 10
X = X.reshape(-1, 1)
N = len(X)
clf = KernelSVR(N, eps=eps, C=C, gamma=50, iters=50000)

In [13]:
clf.fit(X, y)

In [14]:
pred = clf.predict(X)